In [8]:
# well come to our comon envirinment

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv
import math

# ---------------------------------------------------------------------------------------------------
from link_scraper import all_links_scraper

# Create an instance of the class with the desired output directory
scraper = all_links_scraper(output_dir="C:\\Users\\becod\\AI\\my-projects\\Majid_immoeliza_scraping\\Majid_Mustafa\\Immo_scrapping_draft\\Links")

# Scrape the links
filtered_links = scraper.scrape_links()

# Save them to CSV files
scraper.save_links_to_csv(filtered_links)

# ---------------------------------------------------------------------------------------------------





ImportError: cannot import name 'all_links_scraper' from 'link_scraper' (c:\Users\becod\AI\my-projects\Majid_immoeliza_scraping\Majid_mustafa\Immo_scrapping_draft\link_scraper.py)

In [2]:

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd 
df = pd.read_csv("C:\\Users\\mgabi\\Desktop\\becode\\becode_projects\\immoweb_scraping\\all_links.csv")

# Convert the 'url' column to a list (assuming the column is named 'url')
List_url = df['URL'].tolist()

# Print the list of URLs to verify


def request_url(url):
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"}
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.content, "html.parser") 
    property_details = []
     # ID
    try:
        html = soup.find("meta", {'property': "og:url"}).get('content')
        html_list = html.split("/")
        Property_ID = html_list[-1]
    except Exception as e:
        print(f"Error in Property_ID: {e}") 
        Property_ID = None
    # Postal code
    try:
        postal_code = html_list[-2]
    except Exception as e:
        print(f"Error in postal_code : {e}") 
        postal_code = None
    # Locality
    try:
        locality = html_list[-3]
    except Exception as e:
        print(f"Error in locality : {e}") 
        locality = None
    # Price
    try:
        home_meta_info = soup.find_all("div", {'class': 'grid__item desktop--9'})
        price = home_meta_info[0].find("p", {'class': 'classified__price'}).find_all('span', {'class':'sr-only'})[0].text.strip()
        price = re.sub(r'[^\d.,\-+]', '', price)
    except Exception as e:
        print(f"Error in price : {e}") 
        price = None
    # Type of property
    try:
        # Type_of_property = home_meta_info[0].find("h1", {'class': 'classified__title'}).text.strip()[0:11]# the second [0:11] just to return the property type
        # Type_of_property = re.sub(r'\s+', ' ', Type_of_property)
        Type_of_property= html_list[-5]
    except Exception as e:
        print(f"Error in Type_of_property: {e}") 
        Type_of_property = None
    # Number of bed rooms
    try:
        home_prop_info = soup.find_all("div", {'class': 'text-block__body'})[0].find_all("div", {'class': 'overview__column'})
        bed_rooms =home_prop_info[0].find_all("div", {'class': 'overview__item'})[0].find_all('span', {'class':'overview__text'})[0].text.strip()
        bed_rooms = re.search(r'\d+', bed_rooms)
        if bed_rooms:
            bed_rooms = bed_rooms.group()  # Extract the matched number as a string
        else:
            bed_rooms = None 
    except Exception as e:
        print(f"Error in bed_rooms : {e}")
        bed_rooms = None

    try:
        space = home_prop_info[1].find_all("div", {'class': 'overview__item'})[0].find_all('span', {'class':'overview__text'})[0].text.strip()
        space = re.findall(r'\d+', space)[0] # Extract only the digits
    except Exception as e:
        print(f"Error in space: {e}")
        space = None
    # Kitchen
    try:
        kitchen_keywords = ('Kitchen type', 'Type of kitchen')  # as sometimes it has one of these names, I am not using regex here but, it could be used too
        kitchen_th = soup.find('th', string=lambda x: x and x.strip() in kitchen_keywords)
        if kitchen_th:
            kitchen = kitchen_th.find_next_sibling('td').contents[0].strip()
    
            # Now, check for the kitchen types you're interested in
            if kitchen in ('Installed','Installed', 'Hyper equipped', 'USA  Hyper equipped','Semi equipped','USA hyper equipped'):
                kitchen_type = 1 
            else:
                kitchen_type = 0
        else:
            kitchen_type = 0  # Default value if 'Kitchen type' or 'Type of kitchen' not found 
    except Exception as e:
        print(f"Error in kitchen: {e}")
        kitchen_type = None
    # Building cindition
    try:
        building_condition_header = soup.find('th', string=lambda x: x and x.strip() == 'Building condition').find_parent('tr')

        building_condition = building_condition_header.find('td', class_='classified-table__data').contents[0].strip()
    except Exception as e:
        print(f"Error in building_condition: {e}")
        building_condition =None
    # Number of facades
    try:
        facade_keywords = re.compile(r'Number of (frontages|facades)', re.IGNORECASE)
        facades_th = soup.find('th', string=facade_keywords)
        facades = facades_th.find_next_sibling('td').contents[0].strip() if facades_th else None
        if facades:
            Number_of_facades = facades
        else:
            Number_of_facades = None
        
    except Exception as e:
        print(f"Error in Number_of_facades : {e}")
        Number_of_facades = None

    # Furnished
    try:
        Furnished = soup.find('th', string=lambda x: x and x.strip() == 'Furnished').find_next_sibling('td').contents[0].strip()
        if Furnished == 'Yes':
            Furnished = 1
        else : 
            Furnished = 0
    except Exception as e:
        print(f"Error in Furnished: {e}")
        Furnished = 0
    # open fire space
    try:
        Open_fire = soup.find('th', string=lambda x: x and x.strip() == 'How many fireplaces?').find_next_sibling('td').contents[0].strip()
        if Open_fire:
            Open_fire = 1
        else : 
            Open_fire = 0
    except Exception as e:
        print(f"Error in Open_fire: {e}")
        Open_fire = 0
    # Swimming_pool
    
    try:
        Swimming_pool = soup.find('th', string=lambda text: text and 'Swimming pool' in text.strip()).find_next_sibling('td').contents[0].strip()
        
        if Swimming_pool == 'Yes':
            Swimming_pool = 1
        else:
            Swimming_pool = 0

    except Exception as e:
        print(f"Error in Swimming_pool: {e}")
        Swimming_pool = 0
    #Garden	
    try:
        # garden = soup.find('th', string=lambda x: x and x.strip() == 'Garden surface').find_next_sibling('td').contents[0].strip()
        garden = soup.find('th', string=re.compile(r'^Garden.*')).find_next_sibling('td').contents[0].strip()
        if garden:
            garden = garden
        else : 
            garden = None
    except Exception as e:
        print(f"Error in garden: {e}")
        garden = None
    #Terrace
    try:
        Terrace = soup.find('th', string=re.compile(r'^Terrace.*')).find_next_sibling('td').contents[0].strip()
        if Terrace:
            Terrace = Terrace   
        else : 
            Terrace = None

    except Exception as e:
        print(f"Error in Terrace: {e}")
        Terrace = None
        
        
    property_details.append({
        'Property ID': Property_ID,
        'Postal code': postal_code,
        'Locality name': locality,
        'Price': price,
        'Type of property':Type_of_property,
        'Number of rooms': bed_rooms,
        'Living area':space,
        'Equipped kitchen':kitchen_type,
        'State of building': building_condition,
        'Number of facades': Number_of_facades,
        'Furnished':Furnished ,
        'Open fire':Open_fire,
        'Swimming pool':Swimming_pool,
        'Garden (m²)':garden,
        'Terrace (m²)':Terrace
    })
    
    return property_details

# Assuming List_url contains your URLs
all_property_details = []
for url in List_url:
    all_property_details.extend(request_url(url))

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(all_property_details)

# Save the DataFrame to a CSV file
df.to_csv(Majid_dir, index=False, encoding='utf-8')

print("CSV file created")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\mgabi\\Desktop\\becode\\becode_projects\\immoweb_scraping\\all_links.csv'

In [361]:

df = pd.DataFrame(all_links)

# Save the DataFrame to a CSV file
df.to_csv('C:\\Users\\mgabi\\Desktop\\becode\\becode_projects\\immoweb_scraping\\all_links.csv', index=False, encoding='utf-8')

['https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/20236298', 'https://www.immoweb.be/en/classified/apartment/for-sale/antwerpen-2018/2018/20236289', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/seraing/4100/20237641', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gavere/9890/20236163', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gavere/9890/20236149', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/lier/2500/20237655', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/lier/2500/20237328', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/aalst/9300/20236688', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/evere/1140/20234924', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gavere/9890/202362

In [358]:


# Load the CSV file into a DataFrame
df = pd.read_csv("C:\\Users\\mgabi\\Desktop\\becode\\becode_projects\\immoweb_scraping\\all_links.csv")

# Convert the 'url' column to a list (assuming the column is named 'url')
List_url = df['0'].tolist()

# Print the list of URLs to verify


['https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/20236298', 'https://www.immoweb.be/en/classified/apartment/for-sale/antwerpen-2018/2018/20236289', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/seraing/4100/20237641', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gavere/9890/20236163', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gavere/9890/20236149', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/molenbeek-saint-jean/1080/20234667', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/lier/2500/20237655', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/berchem-sainte-agathe/1082/20234591', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/ganshoren/1083/20234626', 'https://www.immoweb.be/en/classified/new-real-estate-project-ap

In [364]:
all_links = []  
session = requests.Session()               
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
response = session.get("https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2", headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for tag_a in soup.find_all('a', class_="card__title-link", href=True):   # extracting all item links paer group page
        all_links.append(tag_a['href'])

In [365]:
all_links

['https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/20236298',
 'https://www.immoweb.be/en/classified/apartment/for-sale/antwerpen-2018/2018/20236289',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/seraing/4100/20237641',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gavere/9890/20236163',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gavere/9890/20236149',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/lier/2500/20237655',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/lier/2500/20237328',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/aalst/9300/20236688',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/evere/1140/20234924',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gavere/98